In [1]:
import time
import csv
import pandas as pd
import argparse
import numpy as np
import random
from multiprocessing import *
import mplleaflet
import matplotlib.pyplot as plt

from lib.OsrmEngine import *
from lib.Agents import *

In [2]:
exe_loc = './osrm-backend-5.6.0/build/osrm-routed'
map_loc = './osrm-backend-5.6.0/greater-london-latest.osrm'

osrm = OsrmEngine(exe_loc, map_loc)
osrm.start_server()

'The routing server "http://0.0.0.0:5000" starts running'

In [3]:
olat = 0.089653
olng = 51.373206
dlat = 0.089282
dlng = 51.350675

N = 100
stime = time.time()
for i in range(N):
    o1 = random.uniform(-0.01, 0.01)
    o2 = random.uniform(-0.01, 0.01)
    d1 = random.uniform(-0.01, 0.01)
    d2 = random.uniform(-0.01, 0.01)
    out = osrm.get_routing(olat+o1, olng+o2, dlat+d1, dlng+d2)
etime = time.time()
print("Avarage running time per request: %.2fms" % ((etime-stime)*1000/N) )

Avarage running time per request: 4.94ms


In [4]:
out = osrm.get_routing(olat, olng, dlat, dlng)
steps = out['legs'][0]['steps']
steps

[{'distance': 134.6,
  'duration': 33.1,
  'geometry': {'coordinates': [[0.089603, 51.373194],
    [0.089608, 51.373185],
    [0.089751, 51.37296],
    [0.089842, 51.372817],
    [0.090346, 51.372133],
    [0.09035, 51.372081]],
   'type': 'LineString'},
  'intersections': [{'bearings': [161],
    'entry': [True],
    'location': [0.089603, 51.373194],
    'out': 0}],
  'maneuver': {'bearing_after': 161,
   'bearing_before': 0,
   'location': [0.089603, 51.373194],
   'type': 'depart'},
  'mode': 'driving',
  'name': 'Station Approach',
  'weight': 33.1},
 {'distance': 125.3,
  'duration': 11.9,
  'geometry': {'coordinates': [[0.09035, 51.372081],
    [0.090681, 51.371981],
    [0.091107, 51.371866],
    [0.091603, 51.371806],
    [0.092071, 51.3718]],
   'type': 'LineString'},
  'intersections': [{'bearings': [120, 300, 345],
    'entry': [True, True, False],
    'in': 2,
    'location': [0.09035, 51.372081],
    'out': 0},
   {'bearings': [105, 195, 300],
    'entry': [True, True, Fa

In [5]:
out = osrm.get_distance_duration(olat, olng, dlat, dlng)
out

(2830.1, 285.9)

In [42]:
class Veh(object):
    """ 
    Veh is a class for vehicles
    Attributes:
        id: sequential unique id
        lat: current latitude
        lng: current longitude
        tlat: target (end of route) latitude
        tlng: target (end of route) longitude
        t: system time at current state
        k: capacity
        n: number of passengers on board
        jobs: a list of jobs in the format of (request id, pickup or dropoff, target lat, target lng)
        route: a list of legs
        c: total cost of the route
        d: total distance of the route
    """
    
    def __init__(self, id, lat=0.089653, lng=51.373206, t=0.0, k=4):
        self.id = id
        self.lat = lat
        self.lng = lng
        self.tlat = lat
        self.tlng = lng
        self.t = t
        self.k = k
        self.n = 0
        self.jobs = []
        self.route = []
        self.c = 0.0
        self.d = 0.0
        
    def get_location(self):
        return (self.lat, self.lng)
    
    def move_to_location(self, lat, lng):
        self.lat = lat
        self.lng = lng
    
    def add_job(self, rid, pd, rlat, rlng):
        self.jobs.append( (rid, pd, rlat, rlng) )
        out = osrm.get_routing(self.tlat, self.tlng, rlat, rlng)
        assert len(out['legs']) == 1
        leg = Leg(out['legs'][0]['distance'], out['legs'][0]['duration'], steps=[])
        for s in range( len(out['legs'][0]['steps']) ):
            step = Step(out['legs'][0]['steps'][s]['distance'],
                        out['legs'][0]['steps'][s]['duration'],
                        out['legs'][0]['steps'][s]['geometry']['coordinates'])
            leg.add_step(step)
        assert len(step.geo) == 2
        assert step.geo[0] == step.geo[1]
        self.add_leg(leg)
        assert len(self.jobs) == len(self.route)
    
    def add_pickup_job(self, req):
        self.add_job(req.id, 1, req.olat, req.olng)
    
    def add_dropoff_job(self, req):
        self.add_job(req.id, -1, req.dlat, req.dlng)

    def add_leg(self, leg):
        self.route.append(leg)
        self.tlat = leg.steps[-1].geo[1][0]
        self.tlng = leg.steps[-1].geo[1][1]
        self.d += leg.d
        self.c += leg.c
        
    def pop_job(self):
        self.jobs.pop(0)
        self.pop_leg()
        assert len(self.jobs) == len(self.route)
        
    def pop_leg(self):
        leg = self.route.pop(0)
        self.d -= leg.d
        self.c -= leg.c
        
    def pop_step(self):
        step = self.route[0].steps.pop(0)
        self.c -= step.c
        self.d -= step.d
        self.route[0].c -= step.c
        self.route[0].d -= step.d
        
    def cut_step(self, pct):
        step = self.route[0].steps[0]
        dis = 0
        sega = step.geo[0]
        for segb in step.geo[1:]:
            dis += np.sqrt( (sega[0] - segb[0])**2 + (sega[1] - segb[1])**2)
            sega = segb
        dis_ = 0
        sega = step.geo[0]
        for segb in step.geo[1:]:
            _dis = np.sqrt( (sega[0] - segb[0])**2 + (sega[1] - segb[1])**2)
            dis_ += _dis
            if dis_ / dis > pct:
                break
            sega = segb
        while step.geo[0] != sega:
            step.geo.pop(0)
        _pct = (pct * dis - dis_ + _dis) / _dis
        
        step.geo[0][0] = sega[0] + _pct * (segb[0] - sega[0])
        step.geo[0][1] = sega[1] + _pct * (segb[1] - sega[1])
        
        self.c -= step.c * pct
        self.d -= step.d * pct
        self.route[0].c -= step.c * pct
        self.route[0].d -= step.d * pct
        self.route[0].steps[0].c -= step.c * pct
        self.route[0].steps[0].d -= step.d * pct  
        
    def move_to_time(self, t):
        dt = t - self.t
        assert dt >= 0
        done = []
        if dt == 0 or len(self.jobs) == 0:
            self.t = t
            return done
        while dt > 0 and len(self.jobs) > 0:
            if self.route[0].c <= dt:
                dt -= self.route[0].c
                self.t += self.route[0].c
                self.move_to_location(self.jobs[0][2], self.jobs[0][3])
                done.append( (self.jobs[0][0], self.jobs[0][1], self.t) )
                self.n += self.jobs[0][1]
                self.pop_job()
            else:
                while dt > 0 and len(self.route[0].steps) > 0:
                    if self.route[0].steps[0].c <= dt:
                        dt -= self.route[0].steps[0].c
                        self.pop_step()
                    else:
                        pct = dt / self.route[0].steps[0].c
                        self.cut_step(pct)
                        self.move_to_location(self.route[0].steps[0].geo[0][0], self.route[0].steps[0].geo[0][1])
                        self.t = t
                        return done
        assert self.n == 0
        assert np.isclose(self.d, 0.0)
        assert np.isclose(self.c, 0.0)
        self.t = t
        self.d = 0.0
        self.c = 0.0
        return done
    
    def draw(self):
        plt.plot(self.lat, self.lng, 'b', marker='o')
        for l in range(len(veh.route)):
            for s in range(len(veh.route[l].steps)):
                step = np.transpose( self.route[l].steps[s].geo )
                plt.plot(step[0], step[1], 'b', linestyle='-' if l == 0 else '--')
                        
    def __str__(self):
        str =  "veh {0} at ({1},{2}) when t = {3}; occupancy = {4}/{5}".format(
            self.id, self.lat, self.lng, self.t, self.n, self.k)
        str += "\n  has {0} job(s), distance = {1}, cost = {2}".format(
            len(self.jobs), self.d, self.c)
        for j in self.jobs:
            str += "\n    {0}: req {1} at ({2},{3})".format("pickup" if j[1] > 0 else "dropoff", j[0], j[2], j[3])
        return str

class Req(object):
    """ 
    Req is a class for requests
    Attributes:
        id: sequential unique id
        olat: origin latitude
        olng: origin longitude
        dlat: destination latitude
        dlng: destination longitude
    """
    def __init__(self, id, olat=0.115662, olng=51.374282, dlat=0.089282, dlng=51.350675):
        self.id = id
        self.olat = olat
        self.olng = olng
        self.dlat = dlat
        self.dlng = dlng
    
    def get_origin(self):
        return (self.olat, self.olng)
    
    def get_destination(self):
        return (self.dlat, self.dlng)
    
    def __str__(self):
        return "req {0} from ({1},{2}) to ({3},{4})".format(self.id, self.olat, self.olng, self.dlat, self.dlng)
    

In [43]:
veh = Veh(0)
req = Req(0)
veh.add_pickup_job(req)
veh.add_dropoff_job(req)
print(veh)
done = veh.move_to_time(50)
print(done)
print(veh)

veh 0 at (0.089653,51.373206) when t = 0.0; occupancy = 0/4
  has 2 job(s), distance = 6431.9, cost = 721.2
    pickup: req 0 at (0.115662,51.374282)
    dropoff: req 0 at (0.089282,51.350675)
[]
veh 0 at (0.09274749514764836,51.37191181374996) when t = 50; occupancy = 0/4
  has 2 job(s), distance = 6116.42342519685, cost = 671.2
    pickup: req 0 at (0.115662,51.374282)
    dropoff: req 0 at (0.089282,51.350675)


In [44]:
fig = plt.figure(figsize=(16,10))
veh.draw()
mplleaflet.display(fig)

In [ ]:
osrm.kill_server()